# 📝 Projeto: Chatbot de Atendimento ao Cliente com Memória (RAG + Chat History)

*   **Autor:** Victor Alexandre
*   **Metodologia:** Aprender Construindo

---

## 1. Objetivo Principal

Construir um assistente virtual inteligente (chatbot) capaz de responder a perguntas sobre uma base de conhecimento específica. O diferencial deste projeto é a implementação de **memória conversacional**, permitindo que o chatbot entenda o contexto de perguntas anteriores e responda a interações de acompanhamento de forma coesa e natural.

---

## 2. Arquitetura da Solução

Utilizamos o padrão **RAG (Retrieval-Augmented Generation)**, que funciona em duas grandes etapas:

1.  **Recuperação (Retrieval):** A pergunta do usuário é usada para buscar os trechos mais relevantes de informação dentro do nosso documento. Isso é feito transformando o texto em vetores numéricos e encontrando os mais similares à pergunta.
2.  **Geração (Generation):** Os trechos recuperados, juntamente com a pergunta original e o histórico da conversa, são enviados a um Grande Modelo de Linguagem (LLM), que gera uma resposta final em linguagem natural.

O fluxo é orquestrado pela biblioteca `LangChain`, conectando uma interface de usuário criada com `Streamlit` a uma base de vetorial `FAISS` e a um LLM.

---

## 3. Fonte de Dados (Dataset)

*   **Documento:** Manual do Usuário do WordPress
*   **Origem:** Universidade Federal de Juiz de Fora (UFJF)
*   **Formato:** PDF
*   **Tamanho:** 33 páginas
*   **Descrição:** Um documento denso, com instruções técnicas, guias passo a passo e imagens, simulando um desafio de atendimento ao cliente do mundo real.

---

## 4. Stack de Tecnologias

| Componente | Tecnologia | Propósito |
| :--- | :--- | :--- |
| **Interface do Usuário** | `Streamlit` | Criar uma aplicação web interativa para o chat. |
| **Orquestração** | `LangChain` | A "cola" que conecta todos os componentes do sistema. |
| **Base de Vetores** | `FAISS` | Armazenar e buscar eficientemente os vetores do documento. |
| **Processamento de PDF** | `PyPDFLoader` | Carregar e extrair o texto do nosso documento base. |
| **Modelos de IA (LLM)** | `Google AI / OpenAI` | Gerar as respostas com base no contexto fornecido. |
| **Ambiente de Dev** | `Google Colab` | Nosso ambiente de desenvolvimento e experimentação. |
| **Deploy (App)** | `Ngrok` | Expor nossa aplicação Streamlit do Colab para a web. |

In [1]:
# CÉLULA 1: INSTALAÇÃO DE TODAS AS DEPENDÊNCIAS (VERSÃO CORRIGIDA 2)
!pip install -q -U langchain langchain-community streamlit langchain_google_genai faiss-cpu pypdf pyngrok google-ai-generativelanguage==0.6.15

In [6]:
# CÉLULA 2: AUTENTICAÇÃO E CONFIGURAÇÃO DA API (VERSÃO CORRIGIDA)

import os
from google.colab import userdata, auth
import google.generativeai as genai

# 1. Pega a API Key dos Segredos do Colab
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = api_key
except userdata.SecretNotFoundError:
    print("ERRO: O segredo 'GOOGLE_API_KEY' não foi encontrado.")
    # Interrompe a execução se a chave não for encontrada
    raise ValueError("Configure o segredo 'GOOGLE_API_KEY' no painel do Colab.")

# 2. Autentica o usuário no ambiente do Colab
#    Este é o passo crucial que resolve o erro de metadados.
#    Uma janela de autenticação do Google pode aparecer. Siga os passos.
try:
    auth.authenticate_user()
    print("✅ Usuário autenticado com sucesso.")
except Exception as e:
    print(f"❌ ERRO: Falha na autenticação do usuário. Detalhe: {e}")
    raise

# 3. Configura a biblioteca do GenAI com a chave
try:
    genai.configure(api_key=api_key)
    print("✅ Biblioteca Google Generative AI configurada com sucesso.")
except Exception as e:
    print(f"❌ ERRO: Falha ao configurar a biblioteca GenAI. Detalhe: {e}")
    raise

✅ Usuário autenticado com sucesso.
✅ Biblioteca Google Generative AI configurada com sucesso.


In [7]:
# CÉLULA 3: CARREGAMENTO E PROCESSAMENTO DO PDF (VERSÃO CORRIGIDA)

from google.colab import drive
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Monta o Google Drive (se já montado, não faz nada)
drive.mount('/content/drive', force_remount=False)

# 2. Define o caminho para o seu arquivo PDF
#    USE O MÉTODO "COPIAR CAMINHO" PARA GARANTIR QUE ESTÁ CORRETO
pdf_path = "/content/drive/MyDrive/Projetos/Projetos - Engenharia de IA/Chatbot de Atendimento ao Cliente com Memória (RAG + Chat History)/manualWP_4.2.pdf" # <-- COLE O CAMINHO COPIADO AQUI

# 3. Carrega o documento PDF
try:
    loader = PyPDFLoader(pdf_path)
    docs_raw = loader.load()
    print(f"Documento de {len(docs_raw)} página(s) carregado com sucesso.")

    # 4. Divide o documento em chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True
    )
    docs_processed = text_splitter.split_documents(docs_raw)

    print(f"O documento foi dividido em {len(docs_processed)} chunks para processamento.")

except Exception as e:
    print(f"Ocorreu um erro ao carregar o PDF: {e}")
    print("\nVERIFIQUE SE O CAMINHO ACIMA ESTÁ 100% CORRETO.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Documento de 30 página(s) carregado com sucesso.
O documento foi dividido em 32 chunks para processamento.


In [8]:
# CÉLULA 4: CRIAÇÃO DA BASE DE VETORES (EMBEDDINGS & FAISS)

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Crie o modelo de embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 2. Crie a base de vetores com o FAISS
print("Criando a base de vetores (isso pode levar alguns segundos)...")
vector_store = FAISS.from_documents(docs_processed, embeddings_model)
print("Base de vetores criada com sucesso!")

# 3. Salve o índice localmente
vector_store.save_local("faiss_index_wordpress")
print("Índice FAISS salvo localmente na pasta 'faiss_index_wordpress'.")

Criando a base de vetores (isso pode levar alguns segundos)...
Base de vetores criada com sucesso!
Índice FAISS salvo localmente na pasta 'faiss_index_wordpress'.


In [11]:
# CÉLULA DE DESCOBERTA: Listando os modelos de IA disponíveis

import google.generativeai as genai

print("Modelos de IA disponíveis para sua API Key:")
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(f"  - {m.name}")

Modelos de IA disponíveis para sua API Key:
  - models/gemini-2.5-pro-preview-03-25
  - models/gemini-2.5-flash-preview-05-20
  - models/gemini-2.5-flash
  - models/gemini-2.5-flash-lite-preview-06-17
  - models/gemini-2.5-pro-preview-05-06
  - models/gemini-2.5-pro-preview-06-05
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash-exp
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-exp-image-generation
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-2.0-flash-preview-image-generation
  - models/gemini-2.0-flash-lite-preview-02-05
  - models/gemini-2.0-flash-lite-preview
  - models/gemini-2.0-pro-exp
  - models/gemini-2.0-pro-exp-02-05
  - models/gemini-exp-1206
  - models/gemini-2.0-flash-thinking-exp-01-21
  - models/gemini-2.0-flash-thinking-exp
  - models/gemini-2.0-flash-thinking-exp-1219
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/learnlm-2.0-flash-experim

In [12]:
# CÉLULA 5: CRIAÇÃO DA CADEIA CONVERSACIONAL (VERSÃO FINAL)

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# CORREÇÃO: Usando o alias estável da sua lista de modelos
llm = ChatGoogleGenerativeAI(model="gemini-pro-latest", temperature=0.3)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

chatbot_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=False
)
print("✅ Chatbot (cadeia conversacional) montado e pronto para uso.")

# --- Teste rápido ---
print("\n--- Teste Rápido no Colab ---")
try:
    question = "Como eu acesso o painel administrativo?"
    response = chatbot_chain.invoke({"question": question})
    print(f"Pergunta: {question}")
    print(f"Resposta: {response['answer']}")
    print("\n✅ Teste rápido concluído com sucesso!")
except Exception as e:
    print(f"❌ ERRO durante o teste rápido: {e}")

✅ Chatbot (cadeia conversacional) montado e pronto para uso.

--- Teste Rápido no Colab ---
Pergunta: Como eu acesso o painel administrativo?
Resposta: Para acessar o painel administrativo do site, você deve digitar o seguinte endereço no seu navegador: `http://www.ufjf.br/seusite/wp-admin/`.

Lembre-se de substituir "seusite" pelo nome do seu site.

Após acessar o endereço, aparecerá uma tela de login onde você deverá inserir seu "Nome de usuário" e "Senha".

✅ Teste rápido concluído com sucesso!


In [16]:
# CÉLULA 6: CRIAÇÃO DO ARQUIVO DA APLICAÇÃO (app.py) (VERSÃO FINAL)
%%writefile app.py

import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS
import os

# --- CONFIGURAÇÃO DA PÁGINA ---
st.set_page_config(page_title="Assistente WordPress", page_icon="🧠", layout="centered")
st.markdown("<h1 style='text-align: center; color: #E8E8E8;'>🧠 Assistente Virtual WordPress</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center; color: #A0A0A0;'>Seu copiloto para tirar dúvidas sobre o manual da UFJF.</p>", unsafe_allow_html=True)
st.divider() # Adiciona uma linha divisória

# --- FUNÇÕES CORE (COM CACHING) ---
@st.cache_resource
def load_components():
    """ Carrega os componentes pesados (LLM, Embeddings, Vector Store) uma única vez. """
    if not os.getenv("GOOGLE_API_KEY"):
        st.error("Chave de API do Google não encontrada. Configure o segredo 'GOOGLE_API_KEY' no Colab.")
        st.stop()

    # CORREÇÃO: Usando o alias estável da sua lista de modelos
    llm = ChatGoogleGenerativeAI(model="gemini-pro-latest", temperature=0.3)

    embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.load_local(
        "faiss_index_wordpress",
        embeddings_model,
        allow_dangerous_deserialization=True
    )
    return llm, vector_store.as_retriever(search_kwargs={"k": 5})

def create_conversational_chain(_llm, _retriever):
    """ Cria a cadeia conversacional com memória. """
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    chain = ConversationalRetrievalChain.from_llm(llm=_llm, retriever=_retriever, memory=memory)
    return chain

# --- INICIALIZAÇÃO DA APLICAÇÃO ---
llm, retriever = load_components()

if 'chain' not in st.session_state:
    st.session_state.chain = create_conversational_chain(llm, retriever)

if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "Olá! Como posso ajudar com o manual do WordPress?"}]

# --- LÓGICA DO CHAT ---
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Qual é a sua dúvida?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        with st.spinner("Analisando o documento..."):
            result = st.session_state.chain.invoke({"question": prompt})
            response = result["answer"]
            st.markdown(response)
            st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting app.py


In [17]:
# CÉLULA 7: EXECUÇÃO DO STREAMLIT COM NGROK (VERSÃO FINAL AUTENTICADA)

from pyngrok import ngrok
from google.colab import userdata
import os

# 1. Pega o authtoken do ngrok dos segredos do Colab
try:
    ngrok_authtoken = userdata.get('ngrok_key')
    # 2. Autentica o ngrok
    ngrok.set_auth_token(ngrok_authtoken)
    print("✅ Authtoken do Ngrok configurado com sucesso.")
except userdata.SecretNotFoundError:
    print("❌ ERRO: O segredo 'ngrok_key' não foi encontrado.")
    raise ValueError("Configure o segredo 'ngrok_key' no painel do Colab.")
except Exception as e:
    print(f"❌ ERRO ao configurar o Ngrok: {e}")
    raise

# Mata qualquer processo do streamlit que possa estar rodando
!killall streamlit

# Define a porta
port = 8501

# Executa o Streamlit em background
os.system(f"streamlit run app.py --server.port {port} &")

# 3. Cria o túnel público (agora autenticado)
try:
    public_url = ngrok.connect(port)
    print("="*60)
    print(f"🚀 SEU CHATBOT ESTÁ NO AR! ACESSE ATRAVÉS DESTE LINK:")
    print(public_url)
    print("="*60)
except Exception as e:
    print(f"❌ ERRO ao criar o túnel do Ngrok: {e}")

✅ Authtoken do Ngrok configurado com sucesso.
🚀 SEU CHATBOT ESTÁ NO AR! ACESSE ATRAVÉS DESTE LINK:
NgrokTunnel: "https://preneural-nonascendently-rosy.ngrok-free.dev" -> "http://localhost:8501"
